In [1]:
import psycopg2

from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime, timedelta
import re
import pytz
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import math as math

In [3]:
cur=conn.cursor()

In [4]:
#removed some AND clauses from cur.execute, issleeve isdeck
# COALESCE(cardquantity,maybecardquantity,1)
cur.execute('''SELECT cardname,carddate,AVG(pshipq) as AVGE,percentile_cont(0.5) WITHIN GROUP (ORDER BY pshipq) as MED,COUNT(*) as VLM 
    FROM (SELECT DISTINCT title,cardname,carddate,(price+shipping)/COALESCE(cardquantity,maybecardquantity,1) as pshipq FROM public.transactions
	WHERE possiblybad IS FALSE 
    AND isboosterbox IS NULL
	AND cardname IN(SELECT DISTINCT(cardname) FROM public.tourninfo)
	AND isfoil IS FALSE
    AND cardlanguage IN('english',NULL)
    AND cardset NOT IN('LEA','LEB','U')
    AND isemblem IS NULL
	AND cardspecial IS NULL
	AND isplaymat IS NULL
    AND isgraded IS NULL
    AND iscommanderdeck IS NULL
    AND isultra IS NULL
    AND (lotis IS FALSE OR lotis IS NULL)
    AND issleeve IS NULL
    AND isdeck IS NULL
    AND iscommanderdeck IS NULL
    AND istoken IS NULL
	AND (othercards IS NULL OR othercards='Hour of Devastation;' OR transplit IS TRUE)
	AND saletype IN('normal','italic')) as foo GROUP BY carddate,cardname ORDER BY cardname,carddate ASC''')
# ,'italic'

In [5]:
rows=cur.fetchall()
cur.close()
conn.close()

In [6]:
df=pd.DataFrame(rows,columns=["cardname","carddate","avg","med","quant"])
df['carddate'] = df['carddate'].astype('datetime64[ns]') 

In [7]:
overalllist=[]
for cardname in df["cardname"].unique():
    coeflist=[]
    regvals=df.loc[df['cardname'] == cardname]
    regvals["carddate"]=regvals["carddate"]-regvals.iloc[0]["carddate"]
    regvals["carddate"]=regvals["carddate"].astype('int64')/86400000000000
    for frac in [1-1/2**i for i in range(0,math.floor(math.log(len(regvals),2)))]:
#     for frac in [0,.5]:
        if len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])<8:
            pass
        else:
#             data[['flow_h','flow_c']].min(axis=1)
            minarr=np.minimum(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["avg"],regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["med"])
            regmin=LinearRegression().fit(np.array(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["carddate"]).reshape(-1, 1),minarr)
            regmed=LinearRegression().fit(np.array(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["carddate"]).reshape(-1, 1),regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["med"])
            regavg=LinearRegression().fit(np.array(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["carddate"]).reshape(-1, 1),regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["avg"])
            medcoef=regmed.coef_[0]
            avgcoef=regavg.coef_[0]
            mincoef=regmin.coef_[0]
            coeflist.append((mincoef,len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])))
            coeflist.append((medcoef,len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])))
            coeflist.append((avgcoef,len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])))
    coeflist.append(cardname)
    overalllist.append(coeflist[::-1])
    
# coefdf=pd.DataFrame(overalllist,columns=["cardname","mecoef1","avgcoef1","medcoef.5","avgcoef.5"])
coefdf=pd.DataFrame(overalllist)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [9]:
coefdf.dropna(thresh=2).sort_values(by=3,axis=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
873,"Urborg, Tomb of Yawgmoth","(-1.1795975530029055, 8)","(-1.1795975530029055, 8)","(-1.1795975530029055, 8)","(-0.01926771561355437, 15)","(-0.01926771561355437, 15)","(-0.01926771561355437, 15)","(0.06361488754818698, 29)","(0.0636241097165201, 29)","(0.0636241097165201, 29)",None,None,None,None,None,None
484,"Liliana, the Last Hope","(-0.18071974520820422, 8)","(-0.18071974520820422, 8)","(-0.18071974520820422, 8)","(-0.06829010939815096, 15)","(-0.06829010939815096, 15)","(-0.06829010939815096, 15)","(-0.05330568023099379, 29)","(-0.05507376501517359, 29)","(-0.05330568023099379, 29)",None,None,None,None,None,None
148,Clever Impersonator,"(-0.14632353361915146, 8)","(-0.14632353361915146, 8)","(-0.14632353361915146, 8)","(-0.05085084176197105, 16)","(-0.0509558840960014, 16)","(-0.05085084176197105, 16)","(-0.003689932260521292, 31)","(-0.0036798883651458824, 31)","(-0.003689932260521292, 31)","(-0.004215910438713584, 62)","(-0.004201272967725487, 62)","(-0.004204065027899752, 62)",None,None,None
853,"Torbran, Thane of Red Fell","(-0.13900368591956125, 9)","(-0.13900368591956125, 9)","(-0.13900368591956125, 9)","(-0.034475589393737585, 17)","(-0.034568949095078676, 17)","(-0.034475589393737585, 17)","(-0.0015295978962523045, 34)","(-0.0015278144831532708, 34)","(-0.0015295978962523045, 34)",None,None,None,None,None,None
566,"Nykthos, Shrine to Nyx","(-0.13824673147176442, 9)","(-0.13824673147176442, 9)","(-0.13824673147176442, 9)","(-0.07327862479233323, 17)","(-0.07169636773046206, 17)","(-0.07169636773046206, 17)","(0.020671835482038502, 34)","(0.020612266897972832, 34)","(0.020612266897972832, 34)","(0.05531986725861134, 68)","(0.05532038411473449, 68)","(0.05532038411473449, 68)",None,None,None
95,Brazen Borrower,"(-0.10451250189619159, 10)","(-0.13568251921882285, 10)","(-0.13568251921882285, 10)","(-0.05764767999474027, 19)","(-0.06801675483371053, 19)","(-0.06801675483371053, 19)","(-0.13636622037664656, 37)","(-0.13832030772183654, 37)","(-0.13832030772183654, 37)","(-0.016074128694643194, 74)","(-0.01657194352610448, 74)","(-0.01657194352610448, 74)",None,None,None
325,Gideon of the Trials,"(-0.10961655616156467, 11)","(-0.10961655616156467, 11)","(-0.10961655616156467, 11)","(0.021577625674460383, 21)","(0.021577625674460383, 21)","(0.021577625674460383, 21)","(0.0746170423444017, 42)","(0.0746170423444017, 42)","(0.0746170423444017, 42)",None,None,None,None,None,None
343,Godless Shrine,"(-0.09445974588541274, 12)","(-0.09716857454391195, 12)","(-0.09716857454391195, 12)","(0.04382250770293969, 23)","(0.043790980259305654, 23)","(0.04504305294720114, 23)","(-0.004152168660372795, 45)","(-0.004068188695310878, 45)","(-0.004174801107729649, 45)","(-0.009701020838747704, 89)","(-0.009382533892252555, 89)","(-0.009452190936436903, 89)",None,None,None
429,"Jace, Vryn's Prodigy","(-0.09393769138326304, 11)","(-0.09393769138326304, 11)","(-0.09393769138326304, 11)",None,None,None,None,None,None,None,None,None,None,None,None
17,"Ajani, Strength of the Pride","(-0.08472244315986482, 8)","(-0.08472244315986482, 8)","(-0.08472244315986482, 8)","(0.02595309174748621, 15)","(0.02595309174748621, 15)","(0.02595309174748621, 15)","(0.03392714127627092, 29)","(0.03392714127627092, 29)","(0.03392714127627092, 29)",None,None,None,None,None,None


In [10]:
regvals

,cardname,carddate,avg,med,quant
17650,Zurgo Bellstriker,0.0,1.5000,1.5000,1
17651,Zurgo Bellstriker,22.0,1.2800,1.2800,1
17652,Zurgo Bellstriker,104.0,1.2800,1.2800,1
17653,Zurgo Bellstriker,109.0,0.8750,0.8750,2
17654,Zurgo Bellstriker,111.0,0.7475,0.7475,1
17655,Zurgo Bellstriker,114.0,0.7375,0.7375,1
17656,Zurgo Bellstriker,116.0,0.8000,0.8000,1
17657,Zurgo Bellstriker,117.0,1.5600,1.5600,1
17658,Zurgo Bellstriker,147.0,1.9800,1.9800,1
17659,Zurgo Bellstriker,155.0,0.7600,0.7600,1
